H


In [4]:
! pip install google-cloud
! pip install google-cloud-bigquery[pandas]
! pip install google-cloud-storage

In [6]:
%load_ext google.cloud.bigquery

In [8]:
%%bigquery prod
select * from meetfresh.dim_product

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
%%bigquery ingredient
select * from meetfresh.dim_ingredient

Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
%%bigquery kiosk
select * from meetfresh.Plano_2022_clean

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
print(prod.shape)
print(ingredient.shape)
print(kiosk.shape)

(128, 8)
(24, 5)
(2797, 16)


In [20]:
ingredient.tail()

,ingredient_id,ingredient_name,gpt_assisted,ingredient_long_description,ingredient_image_url
19,UNK1002,Almond Flakes,False,Thinly shaved almond flakes for added texture ...,https://meetfresh.us/wp-content/uploads/2021/0...
20,TP116,Coco Mochi,False,Coconut flavored mochi,None
21,UNK1003,Coco Sago,False,Soft and bouncy sago,None
22,TP117,Sago,False,Chewy and spongey starch from sago palm,None
23,TP023,Lychee Jelly,True,"Made with the finest lychee essence, this deli...",https://meetfresh.us/wp-content/uploads/2021/0...


In [85]:
import sklearn
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [33]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ingredient["ingredient_long_description"])
features = vectorizer.get_feature_names_out()
print(X.shape)



(24, 147)


In [61]:
ingredient[ingredient['ingredient_name'] == 'Caramel Pudding']

,ingredient_id,ingredient_name,gpt_assisted,ingredient_long_description,ingredient_image_url
5,TP106,Caramel Pudding,False,Caramel-flavored pudding with a silky smooth t...,https://meetfresh.us/wp-content/uploads/2021/0...


In [44]:
X[23].data

array([0.22710745, 0.22710745, 0.22710745, 0.22710745, 0.22710745,
       0.22710745, 0.22710745, 0.22710745, 0.22710745, 0.22710745,
       0.22710745, 0.22710745, 0.22710745, 0.15634108, 0.20098969,
       0.20098969, 0.14641157, 0.1680852 , 0.18245884, 0.13022331,
       0.12343659, 0.1680852 , 0.14641157, 0.13781023, 0.20098969])

In [95]:
cosim = cosine_similarity(X)[:5]
for i in range(len(cosim)):
    sim = list(enumerate(cosim[i]))
    sorted_sim = sorted(sim,key=lambda x:-x[1])[1:]
    front_k = [item[0] for item in sorted_sim[:3]]
    print(i, front_k)
    print(ingredient.iloc[i, 1], "<===>\n", ingredient.iloc[front_k, 1])


0 [2, 9, 23]
Taro Balls <===>
 2             Taro
9     Potaro Balls
23    Lychee Jelly
Name: ingredient_name, dtype: object
1 [0, 9, 18]
Mini Q (Mini Taro Ball) <===>
 0       Taro Balls
9     Potaro Balls
18    Sweet Potato
Name: ingredient_name, dtype: object
2 [18, 0, 9]
Taro <===>
 18    Sweet Potato
0       Taro Balls
9     Potaro Balls
Name: ingredient_name, dtype: object
3 [18, 9, 2]
Taro Paste <===>
 18    Sweet Potato
9     Potaro Balls
2             Taro
Name: ingredient_name, dtype: object
4 [16, 21, 8]
Sesame Rice Balls <===>
 16    Rice Balls
21     Coco Sago
8        Q Mochi
Name: ingredient_name, dtype: object


In [81]:
prod.head()

,product_id,product_name,product_group_name,product_ingredient_name,seasonal,gpt_assisted,product_long_description,product_image_url
0,1017,Icy Taro Ball Signature,SIGNATURE SERIES,"Taro Balls, Taro Paste, Potaro Balls, Boba, Sh...",False,False,Our refreshing Icy Taro Ball Signature is made...,https://meetfresh.us/wp-content/uploads/2022/1...
1,2000,Icy Grass Jelly Signature,SIGNATURE SERIES,"Taro Balls, Grass Jelly Shaved Ice, Grass Jelly",False,False,The Icy Grass Jelly Signature is a customer fa...,https://meetfresh.us/wp-content/uploads/2019/1...
2,21007,Hot Red Bean Soup Signature,SIGNATURE SERIES,"Taro Balls, Sesame Rice Balls, Taro, Red Beans...",False,False,Treat yourself to a bowl of our Hot Red Bean S...,https://meetfresh.us/wp-content/uploads/2021/0...
3,2013,Hot Grass Jelly Soup Signature,SIGNATURE SERIES,"Mini Q, Red Beans, Boba, Grass Jelly Soup",False,False,The Hot Grass Jelly Soup Signature is a great ...,https://meetfresh.us/wp-content/uploads/2021/0...
4,1601,Cold Coco Sago Soup Signature,SIGNATURE SERIES,"Mini Q, Taro, Boba, Coco Mochi, Sago, Coco Sag...",False,False,Treat yourself to a bowl of our Cold Coco Sago...,https://meetfresh.us/wp-content/uploads/2023/0...


In [89]:
prod['product_ingredient_name'].str.split(',', expand=False)

0      [Taro Balls,  Taro Paste,  Potaro Balls,  Boba...
1      [Taro Balls,  Grass Jelly Shaved Ice,  Grass J...
2      [Taro Balls,  Sesame Rice Balls,  Taro,  Red B...
3         [Mini Q,  Red Beans,  Boba,  Grass Jelly Soup]
4      [Mini Q,  Taro,  Boba,  Coco Mochi,  Sago,  Co...
                             ...                        
123                                                 None
124                                                 None
125                                                 None
126                   [Purple Rice Soup,  Almond Flakes]
127                         [Almond Soup,  Tofu Pudding]
Name: product_ingredient_name, Length: 128, dtype: object

In [168]:
prod.head()

,product_id,product_name,product_group_name,product_ingredient_name,seasonal,gpt_assisted,product_long_description,product_image_url,product_ingredient_list
0,1017,Icy Taro Ball Signature,SIGNATURE SERIES,"Taro Balls, Taro Paste, Potaro Balls, Boba, Sh...",False,False,Our refreshing Icy Taro Ball Signature is made...,https://meetfresh.us/wp-content/uploads/2022/1...,"[Taro Balls, Taro Paste, Potaro Balls, Boba..."
1,2000,Icy Grass Jelly Signature,SIGNATURE SERIES,"Taro Balls, Grass Jelly Shaved Ice, Grass Jelly",False,False,The Icy Grass Jelly Signature is a customer fa...,https://meetfresh.us/wp-content/uploads/2019/1...,"[Taro Balls, Grass Jelly Shaved Ice, Grass J..."
2,21007,Hot Red Bean Soup Signature,SIGNATURE SERIES,"Taro Balls, Sesame Rice Balls, Taro, Red Beans...",False,False,Treat yourself to a bowl of our Hot Red Bean S...,https://meetfresh.us/wp-content/uploads/2021/0...,"[Taro Balls, Sesame Rice Balls, Taro, Red B..."
3,2013,Hot Grass Jelly Soup Signature,SIGNATURE SERIES,"Mini Q, Red Beans, Boba, Grass Jelly Soup",False,False,The Hot Grass Jelly Soup Signature is a great ...,https://meetfresh.us/wp-content/uploads/2021/0...,"[Mini Q, Red Beans, Boba, Grass Jelly Soup]"
4,1601,Cold Coco Sago Soup Signature,SIGNATURE SERIES,"Mini Q, Taro, Boba, Coco Mochi, Sago, Coco Sag...",False,False,Treat yourself to a bowl of our Cold Coco Sago...,https://meetfresh.us/wp-content/uploads/2023/0...,"[Mini Q, Taro, Boba, Coco Mochi, Sago, Co..."


In [92]:
vectorizer = CountVectorizer()
prod['product_ingredient_name'].fillna("", inplace=True)
X_prod = vectorizer.fit_transform(prod['product_ingredient_name'])
print(X_prod.shape)

(128, 52)


In [104]:
cosim_prod = cosine_similarity(X_name)[:5]
for i in range(len(cosim_prod)):
    sim = list(enumerate(cosim_prod[i]))
    sorted_sim = sorted(sim,key=lambda x:-x[1])[1:]
    front_k = [item[0] for item in sorted_sim[:5]]
    print(i, front_k)
    print(prod.iloc[i, 1], "<===>\n", prod.iloc[front_k, 1])

0 [6, 7, 5, 99, 20]
Icy Taro Ball Signature <===>
 6               Icy Taro Ball Combo B
7               Icy Taro Ball Combo C
5               Icy Taro Ball Combo A
99              Double Taro Signature
20    Cold Purple Rice Soup Signature
Name: product_name, dtype: object
1 [99, 10, 9, 8, 7]
Icy Grass Jelly Signature <===>
 99      Double Taro Signature
10    Icy Grass Jelly Combo C
9     Icy Grass Jelly Combo B
8     Icy Grass Jelly Combo A
7       Icy Taro Ball Combo C
Name: product_name, dtype: object
2 [15, 16, 14, 6, 23]
Hot Red Bean Soup Signature <===>
 15    Hot Red Bean Soup Combo B
16    Hot Red Bean Soup Combo C
14    Hot Red Bean Soup Combo A
6         Icy Taro Ball Combo B
23     Purple Rice Soup Combo B
Name: product_name, dtype: object
3 [11, 12, 44, 8, 9]
Hot Grass Jelly Soup Signature <===>
 11    Hot Grass Jelly Soup Combo A
12    Hot Grass Jelly Soup Combo B
44                  Miss You Drink
8          Icy Grass Jelly Combo A
9          Icy Grass Jelly Combo B
Nam

In [99]:
vectorizer = TfidfVectorizer()
prod['product_long_description'].fillna("", inplace=True)
X_prod_desc = vectorizer.fit_transform(prod['product_long_description'])
print(X_prod_desc.shape)

(128, 352)


In [103]:
cosim_prod = cosine_similarity(X_prod_desc)[:5]
for i in range(len(cosim_prod)):
    sim = list(enumerate(cosim_prod[i]))
    sorted_sim = sorted(sim,key=lambda x:-x[1])[1:]
    front_k = [item[0] for item in sorted_sim[:5]]
    print(i, front_k)
    print(prod.iloc[i, 1], "<===>\n", prod.iloc[front_k, 1])

0 [6, 7, 5, 26, 28]
Icy Taro Ball Signature <===>
 6             Icy Taro Ball Combo B
7             Icy Taro Ball Combo C
5             Icy Taro Ball Combo A
26          Q Mochi Milk Shaved Ice
28    Taro Paste Volcano Shaved Ice
Name: product_name, dtype: object
1 [8, 10, 9, 3, 13]
Icy Grass Jelly Signature <===>
 8            Icy Grass Jelly Combo A
10           Icy Grass Jelly Combo C
9            Icy Grass Jelly Combo B
3     Hot Grass Jelly Soup Signature
13      Hot Grass Jelly Soup Combo C
Name: product_name, dtype: object
2 [15, 116, 16, 14, 118]
Hot Red Bean Soup Signature <===>
 15     Hot Red Bean Soup Combo B
116    Hot Almond Soup Signature
16     Hot Red Bean Soup Combo C
14     Hot Red Bean Soup Combo A
118      Hot Almond Soup Combo B
Name: product_name, dtype: object
3 [11, 12, 13, 14, 16]
Hot Grass Jelly Soup Signature <===>
 11    Hot Grass Jelly Soup Combo A
12    Hot Grass Jelly Soup Combo B
13    Hot Grass Jelly Soup Combo C
14       Hot Red Bean Soup Combo A
16 

In [154]:
kiosk.head(10)

,group_id,group_name,article,name,gross_amount,net_amount,pieces_sold,quantity_sold,avg_price,discount_amount,avg_discount,reference_amount,ref_pct,tot_pct,year,month
0,None,None,34,Sweet Potato Taro Ball,0.0,0.0,3.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,8
1,None,None,34,Sweet Potato Taro Ball,0.0,0.0,1.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,9
2,None,None,34,Sweet Potato Taro Ball,0.0,0.0,3.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,10
3,None,None,34,Sweet Potato Taro Ball,0.0,0.0,1.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,11
4,None,None,34,Sweet Potato Taro Ball,0.0,0.0,3.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,12
5,None,None,35,Taro Paste,0.0,0.0,10.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,8
6,None,None,35,Taro Paste,0.0,0.0,5.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,9
7,None,None,35,Taro Paste,0.0,0.0,4.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,10
8,None,None,35,Taro Paste,0.0,0.0,4.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,11
9,None,None,35,Taro Paste,0.0,0.0,1.0,None,0.0,0.0,0.0,0.0,0.00,NaN,2022,12


In [126]:
kiosk_cp = kiosk[(kiosk["group_name"].notna()) & 
                 (kiosk['group_name'].str.contains("TOPPING") == False) & 
                 (kiosk['group_name'].str.contains("Free") == False) & 
                 (kiosk['group_name'].str.contains("SPECIAL") == False) &
                 (kiosk['group_name'].str.contains("TIPS") == False)
                ]

In [127]:
kiosk_cp

,group_id,group_name,article,name,gross_amount,net_amount,pieces_sold,quantity_sold,avg_price,discount_amount,avg_discount,reference_amount,ref_pct,tot_pct,year,month
334,1001,TARO BALL-COLD,1001,C Taro Ball #1,70.20,70.20,9.0,None,7.80,0.0,0.00,64.80,92.31,3.30,2022,1
335,1001,TARO BALL-COLD,1001,C Taro Ball #1,39.00,39.00,5.0,None,7.80,0.0,0.00,36.00,92.31,2.62,2022,2
336,1001,TARO BALL-COLD,1001,C Taro Ball #1,62.40,54.60,8.0,None,7.80,-7.8,-0.98,57.60,92.31,2.77,2022,3
337,1001,TARO BALL-COLD,1001,C Taro Ball #1,46.80,46.80,6.0,None,7.80,0.0,0.00,43.20,92.31,2.24,2022,4
338,1001,TARO BALL-COLD,1001,C Taro Ball #1,163.80,163.80,21.0,None,7.80,0.0,0.00,151.20,92.31,4.82,2022,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792,99002,DRINK ADDITION,B20,ADD Milk,2.25,2.25,3.0,None,0.75,0.0,0.00,2.25,100.00 100.00,NaN,2022,6
2793,99002,DRINK ADDITION,B20,ADD Milk,0.75,0.75,1.0,None,0.75,0.0,0.00,0.75,100.00,100.00,2022,11
2794,99003,Waffle Addition,TP206,+Taro Paste (W),1.50,1.50,1.0,None,1.50,0.0,0.00,0.50,33.33 100.00,NaN,2022,6
2795,99999,OPEN DRAWER,99998,Open DRAWER,0.00,0.00,2.0,None,0.00,0.0,0.00,0.00,0.00,NaN,2022,4


In [155]:
kiosk_sum = kiosk_cp[["name", "gross_amount", "net_amount", "pieces_sold", "month"]].groupby(["name", "month"]).sum()
kiosk_avg = kiosk_cp[["name", "gross_amount", "net_amount", "pieces_sold", "avg_price", "month"]].groupby(["name", "month"]).mean()
kiosk_std = kiosk_cp[["name", "gross_amount", "net_amount", "pieces_sold", "month"]].groupby(["name", "month"]).std()

In [156]:
temp = kiosk_sum.sort_values(["gross_amount", "net_amount", "pieces_sold"], ascending=False)
temp.head(20)

gross_amount  net_amount  pieces_sold
name                         month                                       
Grass Jelly Signature-C      6            7339.8     7220.46        941.0
                             5            7051.2     6906.90        904.0
                             7            6815.3     6616.65        813.0
                             8            6230.5     6103.85        733.0
                             4            6146.4     5971.68        788.0
                             1            5740.8     5591.04        736.0
                             9            5397.5     5250.45        635.0
                             3            5280.6     5078.58        677.0
                             10           4700.5     4587.45        553.0
                             2            4126.2     3997.50        529.0
                             12           4020.5     3903.62        473.0
                             11           3927.0     3871.75        462.0
RED BEAN SOUP SIGNATURE- HOT 12           3060.0     3031.95        360.0
                             11           2788.0     2733.60        328.0
Mangoberry Milk Shaved Ice   6            2704.8     2670.30        196.0
                             5            2663.4     2635.80        193.0
Icy Taro Ball Signature      7            2628.3     2606.73        313.0
                             6            2503.8     2480.40        321.0
                             5            2464.8     2435.16        316.0
                             8            2346.0     2337.50        276.0

In [165]:
temp

,level_0,index,name,month,gross_amount,net_amount,pieces_sold
0,0,0,Grass Jelly Signature-C,6,7339.80,7220.46,941.0
1,1,1,Grass Jelly Signature-C,5,7051.20,6906.90,904.0
2,2,2,Grass Jelly Signature-C,7,6815.30,6616.65,813.0
3,3,3,Grass Jelly Signature-C,8,6230.50,6103.85,733.0
4,4,4,Grass Jelly Signature-C,4,6146.40,5971.68,788.0
...,...,...,...,...,...,...,...
2454,2454,2454,OPEN FOOD-NO TAX,7,0.75,0.75,1.0
2455,2455,2455,+Noodle Jelly,7,0.50,0.50,1.0
2456,2456,2456,MF Reusable Straw,9,0.00,0.00,12.0
2457,2457,2457,Open DRAWER,4,0.00,0.00,2.0


In [ ]:
# followup:
    1. Scaling: STD(), MEAN()
    2. Seasonality